<a href="https://colab.research.google.com/github/YoavLevinger/see/blob/main/see-methods/01-software-similarity/colab/01_see_clean_and_load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Direct Download from GitHub

In [2]:
!wget -O sdee_lite.sql "https://raw.githubusercontent.com/YoavLevinger/see/main/docs/02-follow-oss-effort-estimation/dataset/sdee_lite.sql"

--2025-03-21 16:01:03--  https://raw.githubusercontent.com/YoavLevinger/see/main/docs/follow-oss-effort-estimation/dataset/sdee_lite.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-21 16:01:03 ERROR 404: Not Found.



In [ ]:
import re

# Load SQL file
sql_file_path = "sdee_lite.sql"

with open(sql_file_path, "r", encoding="utf-8") as file:
    sql_script = file.read()

# Remove MySQL-specific statements
sql_script = re.sub(r"ENGINE=\w+", "", sql_script)  # Remove ENGINE=
sql_script = re.sub(r"AUTO_INCREMENT", "", sql_script)  # Remove AUTO_INCREMENT
sql_script = re.sub(r"DEFAULT CHARSET=\w+", "", sql_script)  # Remove CHARSET
sql_script = re.sub(r"COLLATE \w+", "", sql_script)  # Remove COLLATE statements
sql_script = re.sub(r"ON UPDATE [^,]+", "", sql_script)  # Remove ON UPDATE constraints
sql_script = re.sub(r"UNSIGNED", "", sql_script)  # Remove UNSIGNED keyword (SQLite does not support it)
sql_script = re.sub(r"COMMENT '.*?'", "", sql_script)  # Remove COMMENTs
sql_script = re.sub(r"LOCK TABLES `?\w+`? WRITE;", "", sql_script, flags=re.IGNORECASE)
sql_script = re.sub(r"UNLOCK TABLES;", "", sql_script, flags=re.IGNORECASE)

# Save the cleaned SQL script
clean_sql_path = "/content/sdee_lite_cleaned.sql"
with open(clean_sql_path, "w", encoding="utf-8") as file:
    file.write(sql_script)

print("SQL file cleaned and saved:", clean_sql_path)


SQL file cleaned and saved: /content/sdee_lite_cleaned.sql


Load the Cleaned SQL File into SQLite

In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite (in-memory database)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Read and execute the cleaned SQL script
with open(clean_sql_path, "r", encoding="utf-8") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

print("Database loaded successfully!")


Database loaded successfully!


# List Available Tables

In [ ]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print(tables)


                  name
0      avg_repo_effort
1             features
2         release_info
3  release_wise_effort
4     repo_info_pv_vec


# Load Data from Key Tables


In [ ]:
# Load vectorized project descriptions
query = "SELECT * FROM repo_info_pv_vec LIMIT 5;"
df_repo_info = pd.read_sql_query(query, conn)
print(df_repo_info.head())

# Load effort estimation data
query = "SELECT * FROM avg_repo_effort LIMIT 5;"
df_effort = pd.read_sql_query(query, conn)
print(df_effort.head())


                  category            owner          repo  \
0  configuration_libraries             juxt          aero   
1  configuration_libraries               pd         figgy   
2  configuration_libraries  kelseyhightower     envconfig   
3  configuration_libraries      JeffGarland      liaw2015   
4  configuration_libraries      taneryilmaz  libconfigini   

                                              pv_vec   cos_sim  
0  [-6.3111968e-03  7.2230858e-04 -4.5987987e-03 ...  1.000000  
1  [ 0.00492741  0.00173693 -0.00801723  0.005146... -0.122843  
2  [ 6.8379391e-04  8.3461199e-03 -3.9898441e-03 ... -0.129983  
3  [ 2.9276814e-03 -7.2899782e-03 -7.2383620e-03 ...  0.240423  
4  [-0.00100779 -0.00632555  0.00106837 -0.003088... -0.061511  
         owner             repo  devCount  activDays  totlDays  modLOC  \
0  0xInfection  TIDoS-Framework         3         12        99  128963   
1         10up     ElasticPress         3          6       363     509   
2         10up       